In [ ]:
import itertools
from functools import cached_property
from typing import Dict

import numpy as np
import pytest
from attrs import frozen

import cirq_qubitization.testing as cq_testing
from cirq_qubitization.bloq_algos.and_bloq import And
from cirq_qubitization.bloq_algos.basic_gates import OneEffect, OneState, ZeroEffect, ZeroState, XGate
from cirq_qubitization.quantum_graph.bloq import Bloq
from cirq_qubitization.quantum_graph.composite_bloq import CompositeBloqBuilder, SoquetT
from cirq_qubitization.quantum_graph.fancy_registers import FancyRegisters
from cirq_qubitization.quantum_graph.quimb_sim import cbloq_to_dense

In [ ]:
from cirq_qubitization.jupyter_tools import show_bloq

In [ ]:
@frozen
class AndIdentity(Bloq):
    @cached_property
    def registers(self) -> 'FancyRegisters':
        return FancyRegisters.build(q0=1, q1=1)

    def build_composite_bloq(
        self, bb: 'CompositeBloqBuilder', q0: 'SoquetT', q1: 'SoquetT'
    ) -> Dict[str, 'SoquetT']:
        qs, trg = bb.add(And(), ctrl=[q0, q1])
        ((q0, q1),) = bb.add(And(adjoint=True), ctrl=qs, target=trg)
        return {'q0': q0, 'q1': q1}


In [ ]:
bloq = AndIdentity()
show_bloq(bloq.decompose_bloq())

In [ ]:
bloq.tensor_contract()

In [ ]:
bloq.decompose_bloq().tensor_contract()

In [ ]:
bb = CompositeBloqBuilder()
q0 = bb.add_register('q0', 1)
q1 = bb.add_register('q1', 1)
q0, q1 = bb.add(AndIdentity(), q0=q0, q1=q1)
q0, q1 = bb.add(AndIdentity(), q0=q0, q1=q1)
cbloq = bb.finalize(q0=q0, q1=q1)
show_bloq(cbloq)

In [ ]:
cbloq.tensor_contract()

In [ ]:
bloq = XGate()
show_bloq(bloq)

In [ ]:
bloq.tensor_contract()

In [ ]:
bb = CompositeBloqBuilder()
q = bb.add_register('q', 1)
q, = bb.add(XGate(), q=q)
q, = bb.add(XGate(), q=q)
xx = bb.finalize(q=q)
show_bloq(xx)

In [ ]:
xx.tensor_contract()

In [ ]:
@frozen
class XNest(Bloq):
    @cached_property
    def registers(self) -> 'FancyRegisters':
        return FancyRegisters.build(r=1)

    def build_composite_bloq(
        self, bb: 'CompositeBloqBuilder', r: 'SoquetT'
    ) -> Dict[str, 'SoquetT']:
        (r,) = bb.add(XGate(), q=r)
        return {'r': r}


@frozen
class XDoubleNest(Bloq):
    @cached_property
    def registers(self) -> 'FancyRegisters':
        return FancyRegisters.build(s=1)

    def build_composite_bloq(
        self, bb: 'CompositeBloqBuilder', s: 'SoquetT'
    ) -> Dict[str, 'SoquetT']:
        (s,) = bb.add(XNest(), r=s)
        return {'s': s}


In [ ]:
XNest().tensor_contract()

In [ ]:
from cirq_qubitization.quantum_graph.quimb_sim import _bloq_defines_add_my_tensors

xx = XDoubleNest()
xx = xx.as_composite_bloq()

cb = xx.flatten(lambda binst: not _bloq_defines_add_my_tensors(binst.bloq))
show_bloq(cb)

In [ ]:
XDoubleNest().tensor_contract()

In [ ]:
show_bloq(XDoubleNest().decompose_bloq())

In [ ]:
self = XDoubleNest().decompose_bloq()

bb, init_soqs = CompositeBloqBuilder.from_registers(self.registers)
binst_map = {}
for binst, soqs in self.iter_bloqsoqs(in_soqs=init_soqs, binst_map=binst_map):
    new_binst, _ = bb.add_2(binst.bloq, **soqs)
    binst_map[binst] = new_binst

bb.finalize(**self.final_soqs(binst_map))